# Training of MGA framework

### INIT

In [1]:
from google.colab import drive
try:
  drive.mount('/content/drive')
except:
  print("can't mount drive. Be sure you are not using a local runtime")
  pass


Mounted at /content/drive


In [19]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/insilicodrug/insilicodrug_dataset_info.csv')
df = df[df.show == 'yes']
df = df[df.roc_prc_train.notna()]
df = df.dropna(axis = 1)
df[df.columns[11:]].mean()

,0
mcc_test,0.574958
mcc_train,0.778580
mcc_val,0.599104
roc_auc_test,68.470568
roc_auc_train,70.955154
roc_auc_val,62.668361
roc_prc_test,0.836300
roc_prc_train,0.956227
roc_prc_val,0.855018


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
%cd /content/drive/MyDrive/Colab\ Notebooks/MGA

/content/drive/MyDrive/Colab Notebooks/MGA


In [4]:
%%capture
%pip install -U torch==2.4.1
#%pip install -U dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html #-f https://data.dgl.ai/wheels/torch-2.4/repo.html #
%pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
%pip install -U dglgo -f https://data.dgl.ai/wheels-test/repo.html
%pip install -U rdkit molvs dgllife

In [5]:
#import os
#os.environ['DGLBACKEND'] = 'tensorflow'

In [6]:
import dgl

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


### Load libraries

In [7]:
import numpy as np
def get_safe_balanced_split(target, trainSize=0.2, getTestIndexes=True, shuffle=False, seed=None, force=False, oversampling = False):
    classes, counts = np.unique(target, return_counts=True)
    nPerClass = float(len(target))*float(trainSize)/float(len(classes)) if force else min(counts)*trainSize

    if force and nPerClass > np.min(counts):
        print("Insufficient data to produce a balanced training data split.")
        print("Classes found %s"%classes)
        print("Classes count %s"%counts)
        ts = float(trainSize*np.min(counts)*len(classes)) / float(len(target))
        print("trainSize is reset from %s to %s"%(trainSize, ts))
        trainSize = ts
        nPerClass = float(len(target))*float(trainSize)/float(len(classes))
    # get number of classes
    nPerClass = int(nPerClass)
    print("Data splitting on %i classes and returning %i per class"%(len(classes),nPerClass ))
    # get indexes
    trainIndexes = []
    for c in classes:
        if seed is not None:
            np.random.seed(seed)
        cIdxs = np.where(target==c)[0]
        cIdxs = np.random.choice(cIdxs, nPerClass, replace=False)
        trainIndexes.extend(cIdxs)
    # get test indexes
    testIndexes = None
    if getTestIndexes:
        testIndexes = list(set(range(len(target))) - set(trainIndexes))
    # shuffle
    if shuffle:
        trainIndexes = random.shuffle(trainIndexes)
        if testIndexes is not None:
            testIndexes = random.shuffle(testIndexes)
    # return indexes
    return trainIndexes, testIndexes

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
def balanced_split(y, test_size, val_size = 0.0, oversampling = False):
    test, train = get_safe_balanced_split(y, trainSize=test_size + val_size)
    val = []
    if test_size/(test_size+val_size) < 1.0:
        val, test = train_test_split(test, test_size=test_size/(test_size+val_size))
    return ['training' if i in train else ('valid' if i in val else 'test') for i in range(len(y))]


### Prepare dataset

In [50]:
import pandas as pd

pd.options.display.max_columns = None

dataset_file = '/content/drive/MyDrive/insilicodrug/insilicodrug_dataset.csv'
df = pd.read_csv(dataset_file)
df.head(20)

,smiles,group,Carcinogenicity,Ames Mutagenicity,Respiratory toxicity,Eye irritation,Eye corrosion,Cardiotoxicity1,Cardiotoxicity10,Cardiotoxicity30,Cardiotoxicity5,CYP1A2,CYP2C19,CYP2C9,CYP2D6,CYP3A4,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,pgp_in,pgp_in_balanced,pgp_sub,bbbp,hia_bin,dili,t12_rat_liver_cat,t12_rat_plasma_cat,t12_human_liver_cat,t12_human_plasma_cat,hia,herg,gastro_disorder(ochem),renal_disorder(ochem),reproductive_tox,skin_sen,neurotoxicity,Hepatotoxicity,CYP2C9_balanced,CYP2D6_balanced,CYP3A4_balanced,CYP1A2_balanced,CYP2C19_balanced,Hepatotoxicity_clean,rep_tox,fdamdd2,f20,f30,skin_disorder(ochem),rat_tox_bin,Acute oral toxicity (LD50),LC50DM,BCF,LC50,IGC50,hia_reg,Human Clearance,mdck,caco_2,vdss,fu_admetlab,fdamdd,cl2,cl_admetlab,fu_log,ppb_lnka,log_caco2,log_mdck,fu_chembl,lc50fm,rat_tox_reg,logD,LD50_rat,log_ld50_rat,LogS,ClogP,vdss_log,LD50_rat_lnka,water_solubility,human_oral_bio
0,B#N,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.394784,NaN
1,B12B3B4B1C234,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.742403,NaN
2,Bc1c(Br)cncc1Br,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Br/C(=N\Nc1nnn[nH]1)c1ccncc1,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Br/C=C/Br,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Br/C=C/Br,training,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.741500,NaN
6,Br/C=C/c1ccccc1,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Br/C=C\Br,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.319500,NaN
8,BrBr,valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
dfs = [pd.concat([df[['smiles', col]],pd.DataFrame({'group': balanced_split(df.iloc[:, 1], 0.2)})], axis = 1).reset_index(drop=True) for col in df.columns[1:]]
df = pd.concat(dfs)
df = df.groupby(['smiles', 'group'], as_index = False).first(skipna=True)
df.to_csv(dataset_file, index=None)

In [56]:
## Check
for col in df.drop(columns = ['smiles', 'group']).columns:
    try:
        assert pd.Series([ len(df[(df.group == g) & (df[col].notna())][col].drop_duplicates()) >= 2 for g in df.group.drop_duplicates().to_list()]).all()
    except:
        print('Not enought sample for endpoint ' + col)


cols = df.drop(columns=['smiles', 'group'])
cla_cols = [col for col in cols if len(df[col].dropna().drop_duplicates()) == 2]
reg_cols = [col for col in cols if len(df[col].dropna().drop_duplicates()) >  2]
#inv_cols = [col for col in cols if len(df[col].dropna().drop_duplicates()) <  2]
#df = df[cla_cols + reg_cols + ['smiles', 'group']]
#df.to_csv('data/dataset_{task_name}_working(copy).csv'.format(task_name = args['task_name']), index = None)

In [54]:
from utils import build_dataset
args={}
!mkdir -p ./data
args['task_name'] = 'insilicodrug'
args['data_name'] = 'insilicodrug'

args['input_csv'] =  dataset_file
args['output_bin'] = f'./data/{args['task_name']}_dataset.bin'
args['output_csv'] = f'./data/{args['task_name']}_dataset_group.csv'

In [62]:
build_dataset.built_data_and_save_for_splited(
        origin_path=args['input_csv'],
        save_path=args['output_bin'],
        group_path=args['output_csv'],
        task_list_selected=None
        )

Output hidden; open in https://colab.research.google.com to view.

### Train Model

In [63]:
import numpy as np
from utils import build_dataset
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.MY_GNN import collate_molgraphs, EarlyStopping, run_a_train_epoch_heterogeneous, \
    run_an_eval_epoch_heterogeneous, set_random_seed, MGA, pos_weight, Meter
import os
import time
import pandas as pd
start = time.time()

In [64]:
# Tensorboard utils
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf

class AverageValueMeter():
    def __init__(self, log = False):
        self.reset()
        self.log = log
    def reset(self):
        self.sum = 0
        self.num = 0

    def add(self, value, num):
        self.sum += value*num
        self.num += num

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None

class TBoardLogger:
    def __init__(self, logdir, code):
        self.meter = AverageValueMeter()
        self.writer = tf.summary.FileWriter(logdir)
        self.code = code
    def add(self, value, n):
        self.meter.add(value, n)
    def write(self, global_step):
        self.writer.add_scalar(self.code, self.meter.value(), global_step=global_step)
    def reset(self):
        self.meter.reset()

class TF2Logger(object):
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        with self.writer.as_default():
            tf.summary.scalar(tag, value, step=step)
            self.writer.flush()
        # summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        # self.writer.add_summary(summary, step)

    def list_of_scalars_summary(self, tag_value_pairs, step):
        """Log scalar variables."""
        with self.writer.as_default():
            for tag, value in tag_value_pairs:
                tf.summary.scalar(tag, value, step=step)
            self.writer.flush()
        # summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value) for tag, value in tag_value_pairs])
        # self.writer.add_summary(summary, step)


import random
def shuffle_df(df):
    group = df.group
    indexes = group.index.to_list()
    group = group.to_list()
    random.shuffle(indexes)
    for i, index in enumerate(indexes):
        df.at[index, 'group'] = group[i]
    return df

In [14]:
# task name (model name)
args['task_name'] = 'sider_dataset'
args['data_name'] = 'sider_dataset'
args['times'] = 6

In [65]:
print('classification tasks:\n\t'+'\n\t'.join(cla_cols))
print('regression tasks:\n\t'+ '\n\t'.join(reg_cols))

classification tasks:
	Carcinogenicity
	Ames Mutagenicity
	Respiratory toxicity
	Eye irritation
	Eye corrosion
	Cardiotoxicity1
	Cardiotoxicity10
	Cardiotoxicity30
	Cardiotoxicity5
	CYP1A2
	CYP2C19
	CYP2C9
	CYP2D6
	CYP3A4
	NR-AR
	NR-AR-LBD
	NR-AhR
	NR-Aromatase
	NR-ER
	NR-ER-LBD
	NR-PPAR-gamma
	SR-ARE
	SR-ATAD5
	SR-HSE
	SR-MMP
	SR-p53
	pgp_in
	pgp_in_balanced
	pgp_sub
	bbbp
	hia_bin
	dili
	t12_rat_liver_cat
	t12_rat_plasma_cat
	t12_human_liver_cat
	t12_human_plasma_cat
	hia
	herg
	gastro_disorder(ochem)
	renal_disorder(ochem)
	reproductive_tox
	skin_sen
	neurotoxicity
	Hepatotoxicity
	CYP2C9_balanced
	CYP2D6_balanced
	CYP3A4_balanced
	CYP1A2_balanced
	CYP2C19_balanced
	Hepatotoxicity_clean
	rep_tox
	fdamdd2
	f20
	f30
	skin_disorder(ochem)
	rat_tox_bin
regression tasks:
	Acute oral toxicity (LD50)
	LC50DM
	BCF
	LC50
	IGC50
	hia_reg
	Human Clearance
	mdck
	caco_2
	vdss
	fu_admetlab
	fdamdd
	cl2
	cl_admetlab
	fu_log
	ppb_lnka
	log_caco2
	log_mdck
	fu_chembl
	lc50fm
	rat_tox_reg
	logD
	LD5

In [17]:
TOX21_TASKS = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']
SIDER_TASKS = ['Hepatobiliary disorders',
 'Metabolism and nutrition disorders',
 'Product issues',
 'Eye disorders',
 'Investigations',
 'Musculoskeletal and connective tissue disorders',
 'Gastrointestinal disorders',
 'Social circumstances',
 'Immune system disorders',
 'Reproductive system and breast disorders',
 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)',
 'General disorders and administration site conditions',
 'Endocrine disorders',
 'Surgical and medical procedures',
 'Vascular disorders',
 'Blood and lymphatic system disorders',
 'Skin and subcutaneous tissue disorders',
 'Congenital, familial and genetic disorders',
 'Infections and infestations',
 'Respiratory, thoracic and mediastinal disorders',
 'Psychiatric disorders',
 'Renal and urinary disorders',
 'Pregnancy, puerperium and perinatal conditions',
 'Ear and labyrinth disorders',
 'Cardiac disorders',
 'Nervous system disorders',
 'Injury, poisoning and procedural complications']

In [66]:
# selected task, generate select task index, task class, and classification_num
args['select_task_list'] = [
                        ## Cla
                            'Carcinogenicity', #No buono
                            'Ames Mutagenicity', 'Respiratory toxicity',
                            'Eye irritation', 'Eye corrosion',
                            #'Cardiotoxicity1', 'Cardiotoxicity5', 'Cardiotoxicity10', 'Cardiotoxicity30',
                            #'CYP1A2',
                            #'CYP2C19',
                            #'CYP2C9', 'CYP2D6', 'CYP3A4',
                            #'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
                            #'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53',

                            'pgp_in','pgp_in_balanced', 'pgp_sub', #Rotti
                            'bbbp',
                            #'hia_bin', # I valori non corrispondono a quelli di admetlab
                            #'Hepatotoxicity',




                            'dili' , #'herg',#OK
                            't12_rat_liver_cat',
                            't12_rat_plasma_cat',
                            't12_human_liver_cat',
                            't12_human_plasma_cat',
                            'hia',
                            'herg',

                            #gastro_disorder(ochem) # Sbilanciato
                            #renal_disorder(ochem)  #Sbilanciato
                            #'reproductive_tox',   # Sbilanciato
                            #'skin_sen',           # Sbilanciato
                            #neurotoxicity         # Sbilanciato
                            #Hepatotoxicity        # No buono. Sblilanciato
                            'CYP2C9_balanced',
                            'CYP2D6_balanced',
                            'CYP3A4_balanced',
                            'CYP1A2_balanced',
                            'CYP2C19_balanced',
                            #'NR-AR_balanced',
                            'Hepatotoxicity_clean',
                            'rep_tox',
                            'fdamdd2',
                            'f20',
                            'f30',
                            'skin_disorder(ochem)',
                            'rat_tox_bin',

                        ## Reg
                            'Acute oral toxicity (LD50)',
                            'LC50DM',
                            'BCF',
                            'LC50',
                            'IGC50', #OK
                            #'hia_reg',
                            # 'Human Clearance',
                            # 'caco_2', 'log_mdck', 'fu_chembl',
                            #'cl',
                            #'vdss',
                            #'lc50fm',          #No buono
                            #'fu_admetlab', #'fdamdd'
                            #'cl2','cl_admetlab'
                            'fu_log',
                            'ppb_lnka',
                            'log_caco2',
                            'logD',
                            #'LD50_rat',        #No buono
                            #'log_ld50_rat'     #No buono
                            #'human_oral_bio'
                            #'fdamdd',
                            #'rat_tox_reg',
                            'LogS','ClogP',
                            'vdss_log',
                            #'LD50_rat_lnka', #No

                            'water_solubility',
                            #'human_oral_bio',
                           ]  # change
#args['select_task_list'] = df.drop(columns=['smiles', 'group']).columns.to_list()
#args['select_task_list'] = SIDER_TASKS
args['select_task_index'] = []
args['classification_num'] = 0
args['regression_num'] = 0
args['all_task_list'] = cla_cols + reg_cols#pd.read_csv('data/toxicity(10).csv').drop(columns=['smiles', 'group']).columns.to_list()  # change

# generate select task index
args['select_task_index'] = [index for index, task in enumerate(args['all_task_list']) if task in args['select_task_list']]

# generate classification_num
args['classification_num'] = len([task for task in args['select_task_list'] if task in cla_cols])
args['regression_num'] = len([task for task in args['select_task_list'] if task in reg_cols])

# generate classification_num
if args['classification_num'] != 0 and args['regression_num'] != 0:
    args['task_class'] = 'classification_regression'
if args['classification_num'] != 0 and args['regression_num'] == 0:
    args['task_class'] = 'classification'
if args['classification_num'] == 0 and args['regression_num'] != 0:
    args['task_class'] = 'regression'
print('Classification task:{}, Regression Task:{}'.format(args['classification_num'], args['regression_num']))
args['bin_path'] = args['output_bin']#'./data/' + args['data_name'] + '.bin'
args['group_path'] = args['output_csv']#'./data/' + args['data_name'] + '_group.csv'

Classification task:28, Regression Task:13


In [67]:
import random as rd
# fix parameters of model
#args = {}
args['device'] = "cuda" if torch.cuda.is_available() else "cpu"
args['atom_data_field'] = 'atom'
args['bond_data_field'] = 'etype'
args['classification_metric_name'] = 'roc_prc'
args['regression_metric_name'] = 'r2'
# model parameter
args['num_epochs'] = 500
args['patience'] = 50
args['batch_size'] = 128
args['mode'] = 'higher'
args['in_feats'] = 40
args['rgcn_hidden_feats'] = [64, 64]
args['classifier_hidden_feats'] = 64
args['rgcn_drop_out'] = 0.3
args['drop_out'] = 0.3
args['lr'] = 4
args['weight_decay'] = 5
args['loop'] = True
args['times'] = 1
args['model_file'] = f'./model/{args['task_name']}_early_stop.pth'
print(f"New model will be saved as {args['model_file']}")

New model will be saved as ./model/insilicodrug_early_stop.pth


In [68]:
## Load datasets
train_set, val_set, test_set, task_number = build_dataset.load_graph_from_csv_bin_for_splited(
        bin_path=args['bin_path'],
        group_path=args['group_path'],
        select_task_index=args['select_task_index']
    )

train_set = [[smi, g.to(args['device']), m, b] for smi, g, m, b in train_set ]
val_set = [[smi, g.to(args['device']), m, b] for smi, g, m, b in val_set ]
test_set = [[smi, g.to(args['device']), m, b] for smi, g, m, b in test_set ]

print("Molecule graph generation is complete !")
train_loader = DataLoader(dataset=train_set,
                              batch_size=args['batch_size'],
                              shuffle=True,
                              collate_fn=collate_molgraphs)

val_loader = DataLoader(dataset=val_set,
                            batch_size=args['batch_size'],
                            shuffle=True,
                            collate_fn=collate_molgraphs)

test_loader = DataLoader(dataset=test_set,
                             batch_size=args['batch_size'],
                             collate_fn=collate_molgraphs)

Molecule graph generation is complete !


In [18]:
## Initialize tensorboard logs
import datetime
from torch.utils.tensorboard import SummaryWriter

folder = './tensorboard/logs/'
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment = args['model_file'].rsplit('/')[-1].split('.',2)[0].replace('(','_').replace(')', '_') + '-' + current_time

writers = [SummaryWriter(folder + task) for task in args['select_task_list']]
labels  = [experiment + '/PRC']*args['classification_num'] + [experiment+'/R2']*args['regression_num']
average_writer = SummaryWriter(folder + experiment + '_score')
def tensorflow_logs(writers, labels, results, step):
    for (writer, label, result) in zip(writers, labels, results):
        writer.add_scalar(label, result, step)

print(folder + experiment)

./tensorboard/logs/tox21_early_stop-20260112-143130


In [ ]:
args['device'] = 'cpu'

In [ ]:
## Train Model
from IPython.display import clear_output
from IPython.core.display import HTML
result_pd = pd.DataFrame(columns=args['select_task_list']+['group'] + args['select_task_list']+['group']
                         + args['select_task_list']+['group'])

print("Train model {file_name}".format(file_name = args['model_file']))

!mkdir -p ./model

for time_id in range(args['times']):

    set_random_seed(2020+time_id)
    df = pd.DataFrame(columns=['epoch','score']+args["select_task_list"])

    pos_weight_np = pos_weight(train_set, classification_num=args['classification_num'])
    loss_criterion_c = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight_np.to(args['device']))
    loss_criterion_r = torch.nn.MSELoss(reduction='none')
    model = MGA(in_feats=args['in_feats'],
                rgcn_hidden_feats=args['rgcn_hidden_feats'],
                n_tasks=task_number,
                rgcn_drop_out=args['rgcn_drop_out'],
                classifier_hidden_feats=args['classifier_hidden_feats'],
                dropout=args['drop_out'],
                loop=args['loop'])
    optimizer = Adam(model.parameters(), lr=10**-args['lr'], weight_decay=10**-args['weight_decay'])
    stopper = EarlyStopping(patience=args['patience'], task_name=args['task_name'], mode=args['mode'], filename=args['model_file'])
    model.to(args['device'])
    #logger = TF2Logger('logs')

    for epoch in range(args['num_epochs']):

        # Run epoch
        run_a_train_epoch_heterogeneous(args, epoch, model, train_loader, loss_criterion_c, loss_criterion_r, optimizer)

        # Validation and early stop
        validation_result = run_an_eval_epoch_heterogeneous(args, model, val_loader)
        val_score = np.mean(validation_result)
        early_stop = stopper.step(val_score, model)
        df = pd.concat([df, pd.DataFrame(data=[[epoch, val_score] + validation_result], columns= ['epoch','score'] + args["select_task_list"])])
        ## Print result
        clear_output(wait=True)
        print("Train model {file_name}".format(file_name = args['model_file']))
        print('***************************************************************************************************')
        print('{}, {}/{} time'.format(args['task_name'], time_id+1, args['times']))
        print('***************************************************************************************************')
        display(HTML(df.reset_index(drop=True).style.highlight_max(color = 'lightgreen', axis = 0).to_html()))

        ##Log result
        #tensorflow_logs(writers, labels, validation_result, epoch)

        if early_stop:
            break

    ##Load best model
    stopper.load_checkpoint(model)

    ## Take results
    test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
    train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
    val_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)

    #average_writer.add_scalare('BestScore/Test', test_score)
    #average_writer.add_scalare('BestScore/Validation', val_score)
    #average_writer.add_scalare('BestScore/Train', train_score)

    # deal with results
    result = train_score + ['training'] + val_score + ['valid'] + test_score + ['test']
    result_pd.loc[time_id] = result

    ## Print results
    print('********************************{}, {}_times_result*******************************'.format(args['task_name'], time_id+1))
    print("training_result:", train_score)
    print("val_result:", val_score)
    print("test_result:", test_score)

,epoch,score,Carcinogenicity,Ames Mutagenicity,Respiratory toxicity,Eye irritation,Eye corrosion,pgp_in,pgp_in_balanced,pgp_sub,bbbp,dili,t12_rat_liver_cat,t12_rat_plasma_cat,t12_human_liver_cat,t12_human_plasma_cat,hia,herg,CYP2C9_balanced,CYP2D6_balanced,CYP3A4_balanced,CYP1A2_balanced,CYP2C19_balanced,Hepatotoxicity_clean,rep_tox,fdamdd2,f20,f30,skin_disorder(ochem),rat_tox_bin,Acute oral toxicity (LD50),LC50DM,BCF,LC50,IGC50,fu_log,ppb_lnka,log_caco2,logD,LogS,ClogP,vdss_log,water_solubility
0,0,-0.884349,0.657074,0.682624,0.572311,0.940628,0.835231,0.570767,0.742381,0.694660,0.868481,0.801564,0.608992,0.570038,0.577081,0.500355,0.917972,0.525935,0.767318,0.504448,0.488389,0.690155,0.657456,0.507341,0.820515,0.561214,0.184251,0.195537,0.831594,0.643975,-1.658000,-3.794400,-0.306500,-2.666000,-0.383400,-2.037400,-0.464500,-38.759300,-2.369700,-0.129600,-0.368100,-0.116300,-1.123400
1,1,-0.234011,0.663913,0.690334,0.642682,0.957973,0.881470,0.676624,0.788787,0.726605,0.750764,0.788209,0.545704,0.596565,0.595691,0.454345,0.973690,0.661522,0.781519,0.558627,0.541228,0.733601,0.732134,0.538097,0.854837,0.617216,0.163777,0.193647,0.878722,0.717052,-0.958700,-1.547000,0.454500,-0.393900,-0.000000,-0.978800,-0.158800,-24.286200,-0.598000,0.396900,0.535700,0.046700,-0.812200
2,2,0.094059,0.727943,0.695276,0.660693,0.964032,0.891081,0.723398,0.832891,0.737086,0.879437,0.786341,0.625729,0.589445,0.637658,0.544851,0.963159,0.643258,0.743354,0.531338,0.568716,0.742113,0.728367,0.561295,0.883175,0.636974,0.142181,0.205230,0.841984,0.729506,-0.472400,-0.350400,0.592900,0.151600,0.067800,-0.498100,0.055900,-15.573500,-0.250600,0.545400,0.657300,0.133600,-0.419600
3,3,0.287390,0.747365,0.702804,0.692918,0.972712,0.908450,0.791255,0.842420,0.723810,0.899023,0.828144,0.630680,0.621354,0.632380,0.513253,0.986069,0.695350,0.796478,0.577857,0.595278,0.745306,0.734126,0.612806,0.893452,0.702761,0.170479,0.211930,0.863085,0.749862,-0.193800,-0.084800,0.593900,0.257800,0.136600,-0.191800,0.224800,-10.117100,0.036300,0.679100,0.773800,0.121300,-0.294500
4,4,0.444418,0.737336,0.700874,0.704985,0.976406,0.893506,0.798235,0.832054,0.719782,0.830576,0.757934,0.661826,0.670486,0.649999,0.567666,0.987663,0.703256,0.813735,0.653787,0.619276,0.753461,0.769827,0.638705,0.895129,0.698974,0.182190,0.205653,0.797177,0.758126,-0.090200,0.248200,0.593900,0.287700,0.066700,0.040900,0.367100,-5.096300,0.269000,0.560100,0.786700,0.316100,-0.107400
5,5,0.469482,0.736408,0.712309,0.717707,0.975556,0.902882,0.792624,0.826735,0.717434,0.901976,0.768366,0.600001,0.660652,0.656157,0.599889,0.987113,0.707048,0.836275,0.616846,0.618402,0.783996,0.769628,0.636000,0.893759,0.658671,0.136936,0.177265,0.816677,0.777963,-0.013600,0.258700,0.580300,0.224600,0.206100,0.059600,0.430500,-4.487700,0.300100,0.589600,0.695400,0.384800,0.035100
6,6,0.539135,0.730640,0.724624,0.716280,0.973910,0.920206,0.825731,0.833747,0.699502,0.663047,0.770087,0.682905,0.667161,0.624709,0.607774,0.980629,0.743678,0.834704,0.621515,0.634539,0.778899,0.766041,0.607665,0.899400,0.668412,0.184319,0.203852,0.841778,0.780073,-0.060400,0.320500,0.663400,0.438900,0.217700,0.067000,0.429300,-1.999700,0.303800,0.540700,0.715300,0.400900,0.081300
7,7,0.582233,0.725776,0.728527,0.726932,0.979154,0.938298,0.819069,0.807283,0.689458,0.803885,0.804579,0.701704,0.649585,0.651996,0.614859,0.984775,0.751260,0.842691,0.664544,0.648249,0.793589,0.784101,0.667004,0.898880,0.673087,0.183969,0.193417,0.797756,0.783924,0.087000,0.328700,0.713300,0.392500,0.144500,0.130100,0.448800,-1.163600,0.451800,0.636300,0.742500,0.494100,0.157200
8,8,0.606346,0.729012,0.730487,0.735735,0.980668,0.941047,0.838266,0.820175,0.696608,0.743324,0.852892,0.687345,0.706707,0.648372,0.681012,0.989420,0.747566,0.845272,0.656822,0.656627,0.790853,0.785388,0.648439,0.909248,0.693200,0.189425,0.175401,0.798810,0.794746,0.053800,0.358400,0.668200,0.423900,0.382900,0.082400,0.485900,-0.615600,0.438800,0.614500,0.763700,0.486500,0.243900
9,9,0.614918,

In [27]:
train_score

[np.float64(0.9012853089244366),
 np.float64(0.9350324336711199),
 np.float64(0.7541030370558234),
 np.float64(0.9177922015249607),
 np.float64(0.9762485094298765),
 np.float64(0.9281401839006114),
 np.float64(0.9941641106545979),
 np.float64(0.7659599883594899),
 np.float64(0.9195167779768934),
 np.float64(0.911357246062211),
 np.float64(0.8480213464755961),
 np.float64(0.9889918400239006),
 np.float64(0.7895671269329237),
 np.float64(0.7747340558576148),
 np.float64(0.9645063361851411),
 np.float64(0.9422583266552077),
 np.float64(0.9927012963968482),
 np.float64(0.7800804426790874),
 np.float64(0.9285133983233149),
 np.float64(0.9501930013644134),
 np.float64(0.9484200085907273),
 np.float64(0.9236179029139995),
 np.float64(0.678214579213728),
 np.float64(0.8637581863018624),
 np.float64(0.9434887360933206),
 np.float64(0.992308604605119),
 np.float64(0.9166692101752181)]

In [23]:
## Evaluate Model

args['task_name'] = 'sider_evaluation'
pos_weight_np = pos_weight(train_set, classification_num=args['classification_num'])
loss_criterion_c = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight_np.to(args['device']))
loss_criterion_r = torch.nn.MSELoss(reduction='none')

model = MGA(in_feats=args['in_feats'], rgcn_hidden_feats=args['rgcn_hidden_feats'],
                n_tasks=task_number, rgcn_drop_out=args['rgcn_drop_out'],
                classifier_hidden_feats=args['classifier_hidden_feats'], dropout=args['drop_out'],
                loop=args['loop'])
optimizer = Adam(model.parameters(), lr=10**-args['lr'], weight_decay=10**-args['weight_decay'])
stopper = EarlyStopping(patience=args['patience'], task_name=args['task_name'], mode=args['mode'], filename=args['model_file'])

##Load best model
stopper.load_checkpoint(model)
model = model.to(args['device'])

## Take results
test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
#val_score = run_an_eval_epoch_heterogeneous(args, model, val_loader)

## Save results to file
df_res = pd.DataFrame({'task': [task for index, task in enumerate(args['all_task_list']) if task in args['select_task_list']],
                       'training_score': train_score,
                       #'valid_score': val_score,
                       'test_score': test_score})

args['classification_metric_name'] = 'roc_auc'
test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
#val_score = run_an_eval_epoch_heterogeneous(args, model, val_loader)

## Add results to dataframe
df_res["train_auc"], df_res["test_auc"] = train_score, test_score

def highlight_lowerthan(s, threshold, column, color = 'yellow'):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] < threshold
    return ['background-color: {color}'.format(color = color) if is_max.any() and column not in reg_cols else '' for v in is_max]

df_res.style.apply(highlight_lowerthan, threshold=0.8, column=['test_auc'], axis=1)

/content/drive/MyDrive/Colab Notebooks/MGA/utils/MY_GNN.py:891: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(self.filename, map_location=to

,task,training_score,test_score,train_auc,test_auc
0,NR-AR,0.502691,0.344445,0.923000,0.766400
1,NR-AR-LBD,0.678313,0.446240,0.938100,0.780600
2,NR-AhR,0.287433,0.138172,0.933200,0.718800
3,NR-Aromatase,0.715925,0.342231,0.978900,0.763600
4,NR-ER,0.521218,0.264064,0.899500,0.694200
5,NR-ER-LBD,0.504475,0.440665,0.894600,0.781500
6,NR-PPAR-gamma,0.572530,0.448647,0.911300,0.819400
7,SR-ARE,0.623850,0.350400,0.912000,0.754300
8,SR-ATAD5,0.652762,0.165481,0.990200,0.703500
9,SR-HSE,0.684677,0.401663,0.929000,0.761000


In [30]:
run_an_eval_epoch_heterogeneous(args, model, test_loader)[:args['classification_num']][0][0]

tensor([0.9479, 0.9724, 0.7363, 0.8475, 0.9739, 0.9702, 0.9463, 0.9306, 0.0236,
        0.9914, 0.9250, 0.9593])

In [48]:
##EVALUATE MODEL
args['task_name'] = 'sider_evaluation'
pos_weight_np = pos_weight(train_set, classification_num=args['classification_num'])
loss_criterion_c = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight_np.to(args['device']))
loss_criterion_r = torch.nn.MSELoss(reduction='none')

model = MGA(in_feats=args['in_feats'], rgcn_hidden_feats=args['rgcn_hidden_feats'],
                n_tasks=task_number, rgcn_drop_out=args['rgcn_drop_out'],
                classifier_hidden_feats=args['classifier_hidden_feats'], dropout=args['drop_out'],
                loop=args['loop'])
optimizer = Adam(model.parameters(), lr=10**-args['lr'], weight_decay=10**-args['weight_decay'])
stopper = EarlyStopping(patience=args['patience'], task_name=args['task_name'], mode=args['mode'], filename=args['model_file'])

##Load best model
stopper.load_checkpoint(model)
model = model.to(args['device'])

## Global scores
test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
#val_score = run_an_eval_epoch_heterogeneous(args, model, val_loader)

## In-depth evaluation
cla_met = ['mcc', 'roc_auc', 'roc_prc']
reg_met = ['rmse', 'mae', 'r2']
df_cla_res = pd.DataFrame({'task': args['select_task_list'][:args['classification_num']]})

for m in cla_met:
    args['classification_metric_name'] = m
    df_cla_res["{m}_test".format(m=m)] = run_an_eval_epoch_heterogeneous(args, model, test_loader)[:args['classification_num']]
    df_cla_res["{m}_train".format(m=m)] = run_an_eval_epoch_heterogeneous(args, model, train_loader)[:args['classification_num']]
    #df_cla_res["{m}_val".format(m=m)]  = run_an_eval_epoch_heterogeneous(args, model, val_loader)[:args['classification_num']]


df_res = df_cla_res

def highlight_lowerthan(s, threshold, column, color = 'yellow'):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] < threshold
    return ['background-color: {color}'.format(color = color) if is_max.any() and column not in reg_cols else '' for v in is_max]

#print(df_res.style.apply(highlight_lowerthan, threshold=0.8, column=['test_auc'], axis=1))
df_res = df_res.set_index('task')
df_res.loc['mean'] = df_res.mean()
df_res = df_res.reset_index(drop=False)
df_res.to_csv(f"/content/drive/MyDrive/insilicodrug/{args['task_name']}_mga_result.csv", index = None)

/content/drive/MyDrive/Colab Notebooks/MGA/utils/MY_GNN.py:891: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(self.filename, map_location=to

In [49]:
df_res

,task,mcc_test,mcc_train,roc_auc_test,roc_auc_train,roc_prc_test,roc_prc_train
0,Hepatobiliary disorders,0.478058,0.633495,0.813700,0.898300,0.773004,0.901285
1,Metabolism and nutrition disorders,0.443085,0.475672,0.812300,0.851800,0.897274,0.935032
2,Product issues,-0.026144,0.702123,0.284500,0.998100,0.014348,0.754103
3,Eye disorders,0.385204,0.538052,0.753100,0.868800,0.835226,0.917792
4,Investigations,0.380340,0.455057,0.793500,0.907800,0.942897,0.976249
5,Musculoskeletal and connective tissue disorders,0.365971,0.457794,0.783500,0.848400,0.907034,0.928140
6,Gastrointestinal disorders,0.241664,0.398876,0.802000,0.941700,0.963688,0.994164
7,Social circumstances,0.191638,0.613669,0.702100,0.951600,0.377983,0.765960
8,Immune system disorders,0.260737,0.399216,0.722200,0.818400,0.898919,0.919517
9,Reproductive system and breast disorders,0.449217,0.616437,0.781300,0.898700,0.797374,0.911357


In [ ]:
result

,task,mcc_test,mcc_train,mcc_val,roc_auc_test,roc_auc_train,roc_auc_val,roc_prc_test,roc_prc_train,roc_prc_val,mae_test,mae_train,mae_val,r2_test,r2_train,r2_val
0,Carcinogenicity,0.543848,0.682352,0.413966,0.813300,0.920200,0.749500,0.819935,0.913679,0.833203,NaN,NaN,NaN,NaN,NaN,NaN
1,Ames Mutagenicity,0.396447,0.696330,0.601777,0.823600,0.918200,0.876400,0.955378,0.921903,0.893668,NaN,NaN,NaN,NaN,NaN,NaN
2,Respiratory toxicity,0.567814,0.778155,0.610303,0.855100,0.970200,0.852400,0.902245,0.979976,0.833742,NaN,NaN,NaN,NaN,NaN,NaN
3,Eye irritation,0.854627,0.883684,0.893159,0.984000,0.987800,0.981600,0.994179,0.995493,0.992605,NaN,NaN,NaN,NaN,NaN,NaN
4,Eye corrosion,0.885197,0.955063,0.936879,0.992700,0.997400,0.991900,0.990160,0.995042,0.985808,NaN,NaN,NaN,NaN,NaN,NaN
5,pgp_in,0.726708,0.709406,0.704422,0.943600,0.964700,0.900100,0.956322,0.990742,0.912415,NaN,NaN,NaN,NaN,NaN,NaN
6,pgp_in_balanced,0.736130,0.880379,0.613717,0.970400,0.984200,0.925500,0.972222,0.983222,0.936671,NaN,NaN,NaN,NaN,NaN,NaN
7,pgp_sub,0.618722,0.740882,0.462050,0.870500,0.950000,0.831200,0.873726,0.972747,0.832928,NaN,NaN,NaN,NaN,NaN,NaN
8,bbbp,0.537459,0.894392,0.837852,0.894300,0.988000,0.977200,0.732662,0.993233,0.953218,NaN,NaN,NaN,NaN,NaN,NaN
9,Hepatotoxicity,0.318034,0.674062,0.502613,0.730300,0.930400,0.880700,0.671673,0.958669,0.916411,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
result.to_csv('insilico_drug_mga_result.csv', index = None)

In [ ]:
result.drop(columns = ['task']).mean()

,0
mcc_test,0.320449
mcc_train,0.514134
roc_auc_test,0.757889
roc_auc_train,0.903267
roc_prc_test,0.738526
roc_prc_train,0.905502


## Train Classification Only

In [ ]:
args['select_task_list'] = [col for col in args['select_task_list'] if col in cla_cols]
args['all_task_list'] = cla_cols + reg_cols#pd.read_csv('data/toxicity(10).csv').drop(columns=['smiles', 'group']).columns.to_list()  # change

# generate select task index
args['select_task_index'] = [index for index, task in enumerate(args['all_task_list']) if task in args['select_task_list']]

# generate classification_num
args['classification_num'] = len([task for task in args['select_task_list'] if task in cla_cols])
args['regression_num'] = len([task for task in args['select_task_list'] if task in reg_cols])

# generate classification_num
if args['classification_num'] != 0 and args['regression_num'] != 0:
    args['task_class'] = 'classification_regression'
if args['classification_num'] != 0 and args['regression_num'] == 0:
    args['task_class'] = 'classification'
if args['classification_num'] == 0 and args['regression_num'] != 0:
    args['task_class'] = 'regression'
print('Classification task:{}, Regression Task:{}'.format(args['classification_num'], args['regression_num']))
args['bin_path'] = args['output_bin']#'./data/' + args['data_name'] + '.bin'
args['group_path'] = args['output_csv']#'./data/' + args['data_name'] + '_group.csv'

Classification task:28, Regression Task:0


In [ ]:
import random as rd
# fix parameters of model
#args = {}
args['device'] = "cuda" if torch.cuda.is_available() else "cpu"
args['atom_data_field'] = 'atom'
args['bond_data_field'] = 'etype'
args['classification_metric_name'] = 'roc_prc'
args['regression_metric_name'] = 'r2'
# model parameter
args['num_epochs'] = 500
args['patience'] = 50
args['batch_size'] = 128
args['mode'] = 'higher'
args['in_feats'] = 40
args['rgcn_hidden_feats'] = [64, 64]
args['classifier_hidden_feats'] = 64
args['rgcn_drop_out'] = 0.3
args['drop_out'] = 0.3
args['lr'] = 4
args['weight_decay'] = 5
args['loop'] = True
args['times'] = 1
args['model_file'] = './model/insilicodrug_cla_early_stop.pth'
print(args['model_file'])

./model/insilicodrug_cla_early_stop.pth


In [ ]:
## Initialize tensorboard logs
import datetime
from torch.utils.tensorboard import SummaryWriter

folder = './tensorboard/logs/'
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment = args['model_file'].rsplit('/')[-1].split('.',2)[0].replace('(','_').replace(')', '_') + '-' + current_time

writers = [SummaryWriter(folder + task) for task in args['select_task_list']]
labels  = [experiment + '/PRC']*args['classification_num'] + [experiment+'/R2']*args['regression_num']
average_writer = SummaryWriter(folder + experiment + '_score')
def tensorflow_logs(writers, labels, results, step):
    for (writer, label, result) in zip(writers, labels, results):
        writer.add_scalar(label, result, step)

print(folder + experiment)

./tensorboard/logs/insilicodrug_cla_early_stop-20230324-073927


In [ ]:
## Load datasets
train_set, val_set, test_set, task_number = build_dataset.load_graph_from_csv_bin_for_splited(
        bin_path=args['bin_path'],
        group_path=args['group_path'],
        select_task_index=args['select_task_index']
    )
print("Molecule graph generation is complete !")
train_loader = DataLoader(dataset=train_set,
                              batch_size=args['batch_size'],
                              shuffle=True,
                              collate_fn=collate_molgraphs)

val_loader = DataLoader(dataset=val_set,
                            batch_size=args['batch_size'],
                            shuffle=True,
                            collate_fn=collate_molgraphs)

test_loader = DataLoader(dataset=test_set,
                             batch_size=args['batch_size'],
                             collate_fn=collate_molgraphs)

Molecule graph generation is complete !


In [ ]:
## Train Model
from IPython.display import clear_output
from IPython.core.display import HTML
result_pd = pd.DataFrame(columns=args['select_task_list']+['group'] + args['select_task_list']+['group']
                         + args['select_task_list']+['group'])

print("Train model {file_name}".format(file_name = args['model_file']))


for time_id in range(args['times']):

    set_random_seed(2020+time_id)
    df = pd.DataFrame(columns=['epoch','score']+args["select_task_list"])

    pos_weight_np = pos_weight(train_set, classification_num=args['classification_num'])
    loss_criterion_c = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight_np.to(args['device']))
    loss_criterion_r = torch.nn.MSELoss(reduction='none')
    model = MGA(in_feats=args['in_feats'],
                rgcn_hidden_feats=args['rgcn_hidden_feats'],
                n_tasks=task_number,
                rgcn_drop_out=args['rgcn_drop_out'],
                classifier_hidden_feats=args['classifier_hidden_feats'],
                dropout=args['drop_out'],
                loop=args['loop'])
    optimizer = Adam(model.parameters(), lr=10**-args['lr'], weight_decay=10**-args['weight_decay'])
    stopper = EarlyStopping(patience=args['patience'], task_name=args['task_name'], mode=args['mode'], filename=args['model_file'])
    model.to(args['device'])
    #logger = TF2Logger('logs')

    for epoch in range(args['num_epochs']):

        # Run epoch
        run_a_train_epoch_heterogeneous(args, epoch, model, train_loader, loss_criterion_c, loss_criterion_r, optimizer)

        # Validation and early stop
        validation_result = run_an_eval_epoch_heterogeneous(args, model, val_loader)
        val_score = np.mean(validation_result)
        early_stop = stopper.step(val_score, model)
        df = df.append(pd.DataFrame(data=[[epoch, val_score] + validation_result], columns= ['epoch','score'] + args["select_task_list"]))

        ## Print result
        clear_output(wait=True)
        print("Train model {file_name}".format(file_name = args['model_file']))
        print('***************************************************************************************************')
        print('{}, {}/{} time'.format(args['task_name'], time_id+1, args['times']))
        print('***************************************************************************************************')
        display(HTML(df.reset_index(drop=True).style.highlight_max(color = 'lightgreen', axis = 0).to_html()))

        ##Log result
        tensorflow_logs(writers, labels, validation_result, epoch)

        if early_stop:
            break

    ##Load best model
    stopper.load_checkpoint(model)

    ## Take results
    test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
    train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
    val_score = run_an_eval_epoch_heterogeneous(args, model, val_loader)

    average_writer.add_scalare('BestScore/Test', test_score)
    average_writer.add_scalare('BestScore/Validation', val_score)
    average_writer.add_scalare('BestScore/Train', train_score)

    # deal with results
    result = train_score + ['training'] + val_score + ['valid'] + test_score + ['test']
    result_pd.loc[time_id] = result

    ## Print results
    print('********************************{}, {}_times_result*******************************'.format(args['task_name'], time_id+1))
    print("training_result:", train_score)
    print("val_result:", val_score)
    print("test_result:", test_score)

Train model ./model/insilicodrug_cla_early_stop.pth
***************************************************************************************************
insilicodrug_dataset, 1/1 time
***************************************************************************************************


,epoch,score,Carcinogenicity,Ames Mutagenicity,Respiratory toxicity,Eye irritation,Eye corrosion,pgp_in,pgp_in_balanced,pgp_sub,bbbp,dili,t12_rat_liver_cat,t12_rat_plasma_cat,t12_human_liver_cat,t12_human_plasma_cat,hia,herg,CYP2C9_balanced,CYP2D6_balanced,CYP3A4_balanced,CYP1A2_balanced,CYP2C19_balanced,Hepatotoxicity_clean,rep_tox,fdamdd2,f20,f30,skin_disorder(ochem),rat_tox_bin
0,0,0.646873,0.738437,0.649723,0.513179,0.942965,0.856342,0.752576,0.839162,0.784642,0.686958,0.657062,0.548437,0.611514,0.548157,0.490620,0.973762,0.612698,0.760300,0.510390,0.519299,0.687930,0.691007,0.600759,0.824441,0.583289,0.160158,0.352423,0.578161,0.638060
1,1,0.674723,0.769766,0.671080,0.595697,0.958770,0.872935,0.762432,0.845447,0.771178,0.672889,0.724831,0.499958,0.643564,0.653250,0.569585,0.973647,0.671914,0.802279,0.548231,0.540579,0.723852,0.723297,0.627970,0.837936,0.650072,0.171837,0.284195,0.644462,0.680581
2,2,0.699002,0.765250,0.700004,0.648341,0.963818,0.908846,0.767410,0.847553,0.766406,0.814070,0.748985,0.512319,0.638171,0.685233,0.601605,0.972384,0.706008,0.835526,0.577123,0.563428,0.747813,0.754371,0.634057,0.872776,0.636385,0.212543,0.295949,0.686696,0.709000
3,3,0.713260,0.751698,0.729796,0.674379,0.966786,0.909362,0.800907,0.848253,0.760638,0.781528,0.753472,0.599231,0.661899,0.671790,0.569471,0.981140,0.731879,0.852865,0.575192,0.593394,0.776305,0.796807,0.672974,0.879458,0.631852,0.207387,0.275975,0.782653,0.734184
4,4,0.724905,0.743374,0.752310,0.676984,0.969413,0.904539,0.811116,0.830461,0.757818,0.815854,0.767232,0.596228,0.666835,0.702592,0.619301,0.981311,0.742017,0.859493,0.588225,0.607475,0.791214,0.812604,0.697072,0.886072,0.621526,0.262042,0.302185,0.790875,0.741186
5,5,0.732968,0.769815,0.768256,0.697899,0.978991,0.913860,0.823968,0.838548,0.738832,0.823465,0.804408,0.616343,0.687363,0.717032,0.621144,0.981662,0.750794,0.866646,0.598114,0.624236,0.806922,0.828414,0.690097,0.891329,0.610589,0.238721,0.301474,0.787843,0.746348
6,6,0.737096,0.768787,0.777231,0.699999,0.981395,0.918625,0.831214,0.837752,0.746416,0.795634,0.809293,0.617311,0.701914,0.702448,0.597144,0.981934,0.754105,0.871692,0.605604,0.634258,0.812923,0.836470,0.704154,0.895988,0.654272,0.233498,0.317077,0.801623,0.749934
7,7,0.744113,0.747854,0.791342,0.716366,0.983627,0.933198,0.827584,0.841211,0.706104,0.830901,0.818763,0.599093,0.700850,0.726580,0.626483,0.983930,0.763205,0.877297,0.612467,0.649111,0.816564,0.856047,0.742428,0.897984,0.677579,0.234147,0.334628,0.781168,0.758650
8,8,0.753816,0.748458,0.797614,0.724318,0.984835,0.938629,0.838667,0.847032,0.739291,0.862126,0.822290,0.616171,0.721504,0.752860,0.620324,0.984828,0.764321,0.882332,0.614224,0.659016,0.820285,0.862702,0.751845,0.900531,0.704630,0.242603,0.343382,0.801864,0.760178
9,9,0.757178,0.759613,0.801953,0.726588,0.986511,0.941243,0.834941,0.853057,0.723681,0.848780,0.831900,0.620387,0.743611,0.763985,0.617749,0.983719,0.770920,0.883372,0.640984,0.665113,0.816933,0.861544,0.744898,0.901436,0.713326,0.236059,0.349026,0.818571,0.761080


AttributeError: 'SummaryWriter' object has no attribute 'add_scalare'

## Tox21

In [ ]:
import pandas as pd

pd.options.display.max_columns = None

dataset_file = '/content/drive/MyDrive/insilicodrug/insilicodrug_dataset.csv'
df = pd.read_csv(dataset_file)
df.head(20)

,smiles,group,Carcinogenicity,Ames Mutagenicity,Respiratory toxicity,Eye irritation,Eye corrosion,Cardiotoxicity1,Cardiotoxicity10,Cardiotoxicity30,Cardiotoxicity5,CYP1A2,CYP2C19,CYP2C9,CYP2D6,CYP3A4,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,pgp_in,pgp_in_balanced,pgp_sub,bbbp,hia_bin,dili,t12_rat_liver_cat,t12_rat_plasma_cat,t12_human_liver_cat,t12_human_plasma_cat,hia,herg,gastro_disorder(ochem),renal_disorder(ochem),reproductive_tox,skin_sen,neurotoxicity,Hepatotoxicity,CYP2C9_balanced,CYP2D6_balanced,CYP3A4_balanced,CYP1A2_balanced,CYP2C19_balanced,Hepatotoxicity_clean,rep_tox,fdamdd2,f20,f30,skin_disorder(ochem),rat_tox_bin,Acute oral toxicity (LD50),LC50DM,BCF,LC50,IGC50,hia_reg,Human Clearance,mdck,caco_2,vdss,fu_admetlab,fdamdd,cl2,cl_admetlab,fu_log,ppb_lnka,log_caco2,log_mdck,fu_chembl,lc50fm,rat_tox_reg,logD,LD50_rat,log_ld50_rat,LogS,ClogP,vdss_log,LD50_rat_lnka,water_solubility,human_oral_bio
0,B#N,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.394784,NaN
1,B12B3B4B1C234,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.742403,NaN
2,Bc1c(Br)cncc1Br,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Br/C(=N\Nc1nnn[nH]1)c1ccncc1,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Br/C=C/Br,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Br/C=C/Br,training,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.741500,NaN
6,Br/C=C/c1ccccc1,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Br/C=C\Br,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.319500,NaN
8,BrBr,valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
dfs = [pd.concat([df[['smiles', col]],pd.DataFrame({'group': balanced_split(df.iloc[:, 1], 0.2)})], axis = 1).reset_index(drop=True) for col in df.columns[1:]]
df = pd.concat(dfs)
df = df.groupby(['smiles', 'group'], as_index = False).first(skipna=True)
df.to_csv(dataset_file, index=None)

Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
Data splitting on 3 classes and returning 2678 per class
